# Ejemplo 1: Procesador de mensajes
En el presente ejemplo se implementará un sistema multi agente haciendo uso del patron de Agentes Concurrentes. Por tal motivo, se crearán los siguientes agentes que  tendrán como input inicial un mensaje del usuario:

- ModerationAgent: Detecta que el mensaje no tenga lenguaje ofensivo.
- IntentionAnalyzerAgent: Determina si el mensaje es neutral/petición de ayuda
- ResponseAgent: Genera una respuesta al mensaje del usuario.
- SummarizerAgent: Resume el contenido del mensaje del usuario.
- KeywordExtractorAgent: Extrae palabras clave.
- TranslatorAgent: Traduce el mensaje a inglés.

In [2]:
# Load environment variables

from dotenv import load_dotenv

load_dotenv()

True

In [54]:
# Import necessary dependencies

import asyncio
from dataclasses import dataclass

from autogen_core import (
    DefaultTopicId,
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    default_subscription,
    message_handler,
)

In [55]:
# Defining models for Task and TaskResponse.

@dataclass
class Task:
    task_id: str
    content: str


@dataclass
class TaskResponse:
    task_id: str
    result: str

In [56]:
# Instantiating openai client
import os
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [57]:
# Defining LLM calls of each agent

import json
from typing import Literal, List
from pydantic import BaseModel

async def moderate_message(message):
    """Agente de moderación de contenido"""
    moderation_prompt = f"Evalúa este mensaje y di si contiene contenido ofensivo, spam o prohibido:\n\n'{message}'"
    messages = [
        UserMessage(content=moderation_prompt, source="user"),
    ]
    response = await model_client.create(messages=messages)
    return response.content


async def analyze_intention(message):
    """Agente de análisis de Intención"""

    class IntentionAgentResponse(BaseModel):
      intention: Literal["Neutro", "Solicitud de Ayuda"]

    model_custom = OpenAIChatCompletionClient(
        model="gpt-4o-mini",
        api_key=os.environ.get("OPENAI_API_KEY"),
        response_format=IntentionAgentResponse
    )

    intention_prompt = f"Analiza la intención del siguiente mensaje (Neutro / Solicitud de Ayuda ):\n\n'{message}'"
    messages = [
        UserMessage(content=intention_prompt, source="user"),
    ]
    response = await model_custom.create(messages=messages)
    response = json.loads(response.content)
    return response["intention"]


async def generate_response(message):
    """Agente de generación de respuesta automática"""
    response_prompt = f"Genera una respuesta apropiada para el siguiente mensaje:\n\n'{message}'"
    messages = [
        UserMessage(content=response_prompt, source="user"),
    ]
    response = await model_client.create(messages=messages)
    return response.content


async def extract_keywords(message):
    """Agente de extracción de palabras clave"""

    class KeywordsAgentResponse(BaseModel):
      keywords: List[str]

    model_custom = OpenAIChatCompletionClient(
        model="gpt-4o-mini",
        api_key=os.environ.get("OPENAI_API_KEY"),
        response_format=KeywordsAgentResponse
    )

    keyword_prompt = f"Extrae las palabras clave del siguiente mensaje:\n\n'{message}'"
    messages = [
        UserMessage(content=keyword_prompt, source="user"),
    ]

    response = await model_custom.create(messages=messages)
    response = json.loads(response.content)
    return response["keywords"]


async def summarize_message(message):
    """Agente de resumen del mensaje"""
    summary_prompt = f"Resume este mensaje en una oración breve:\n\n'{message}'"
    messages = [
        UserMessage(content=summary_prompt, source="user"),
    ]
    response = await model_client.create(messages=messages)
    return response.content


async def translate_message(message):
    """Agente de traducción a inglés"""
    translate_prompt = f"Traduce este mensaje al inglés:\n\n'{message}'"
    messages = [
        UserMessage(content=translate_prompt, source="user"),
    ]
    response = await model_client.create(messages=messages)
    return response.content

In [58]:
# Defining agents behavior
agents = [
  {
    "description": "Moderation Agent",
    "func": moderate_message
  },
  {
    "description": "Intention Analyzer Agent",
    "func": analyze_intention
  },
  {
    "description": "Response Agent",
    "func": generate_response
  },
  {
    "description": "Summarizer Agent",
    "func": summarize_message
  },
  {
    "description": "Keyword Extractor Agent",
    "func": extract_keywords
  },
  {
    "description": "Translator Agent",
    "func": translate_message
  },
]

# Defining a Processor in the default subscription

@default_subscription
class Processor(RoutedAgent):
    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> None:
        print(f"{self._description} starting task {message.task_id}")
        agent = list(filter(lambda x: x["description"] == self._description, agents))[0]
        result = await agent["func"](message.content)
        print(f"{self._description} finished task {message.task_id} with response: {result}")

In [59]:
# Registering agents

runtime = SingleThreadedAgentRuntime()

await Processor.register(runtime, "intention_agent", lambda: Processor("Intention Analyzer Agent"))
await Processor.register(runtime, "response_agent", lambda: Processor("Response Agent"))
await Processor.register(runtime, "summarizer_agent", lambda: Processor("Summarizer Agent"))
await Processor.register(runtime, "keyword_agent", lambda: Processor("Keyword Extractor Agent"))
await Processor.register(runtime, "translator_agent", lambda: Processor("Translator Agent"))

runtime.start()

await runtime.publish_message(Task(task_id="task-1", content="Necesito ayuda con python, especificamente con comprension de listas"), topic_id=DefaultTopicId())

await runtime.stop_when_idle()

Intention Analyzer Agent starting task task-1
Response Agent starting task task-1
Summarizer Agent starting task task-1
Keyword Extractor Agent starting task task-1
Translator Agent starting task task-1
Intention Analyzer Agent finished task task-1 with response: Solicitud de Ayuda
Summarizer Agent finished task task-1 with response: Busco ayuda con la comprensión de listas en Python.
Translator Agent finished task task-1 with response: "I need help with Python, specifically with list comprehension."
Keyword Extractor Agent finished task task-1 with response: ['ayuda', 'python', 'comprensión', 'listas', 'programación']
Response Agent finished task task-1 with response: ¡Claro! La comprensión de listas en Python es una forma concisa y eficiente de crear listas. Te permite generar una nueva lista a partir de una existente, aplicando una expresión y un potencial filtrado en una sola línea. 

Aquí tienes un ejemplo básico:

```python
# Supongamos que tienes una lista de números y deseas cr